## `__call__()` Function

`__call__()` 如同 `__init__()`，是 **Python的保留函式**

`__call__()` 使得實體可以像函式一樣被呼叫。

In [1]:
class TestClass:
    def __init__(self):
        print("Test Class has been initialized!")
    def __call__(self, name:str):
        print(f"My name is {name}!")

In [2]:
test_class = TestClass()

Test Class has been initialized!


`test_class`是一個實體，隸屬於 `TestClass`類別。

In [3]:
test_class("allen")

My name is allen!


---


## 實做第一個 Pytorch Model

In [1]:
import os
import torch
from torch import nn

使用 `Pytorch` 前，先確認自己目前使用的裝置是 CPU/GPU:

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### 定義模型

定義 `Pytorch` 的模型架構時，基本上是透過**定義一個類別**來完成。

其中有兩個 Function 是必備的: 

+ `__init__()` : 建構子，通常模型會用到的每一層Layer都需要先在其中被定義。
 + `forward()` : 必需要Override，定義模型如何進行 Forward Propagation。

In [10]:
class NeuralNetwork(nn.Module): # 必須要繼承nn.Module
    def __init__(self): # 建構子，通常裡面會宣告模型中會使用的每一層Layer
        super(NeuralNetwork, self).__init__() # 繼承
        #*以下nn開頭皆為封裝好的Layer
        self.flatten = nn.Flatten() 
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x): # 定義模型的 Forward 流程
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### Forward Propagation

在 `Pytorch` 及 `transformers` 中，預設將 `__call__()` 設定為 `forward()`

也就是說，當你看到 : 

```python
output = model(variable)
```

實際上等同於 : 

```python
output = model.forward(variable)
```

In [34]:
X = torch.rand(1, 28, 28, device=device)


In [35]:
logits = model(X) # 將 X 這個張量送入模型中，進行 Forward Propagation

In [36]:
logits

tensor([[0.0296, 0.0281, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

In [37]:
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


Loss function 透過比較模型的「輸出值」與「真實答案」來衡量模型的好壞:

In [63]:
answer_mock = torch.tensor([[0,1,0,0,0,0,0,0,0,0]], device=device, dtype=torch.float32)
loss = torch.nn.functional.binary_cross_entropy(pred_probab, answer_mock)
loss

tensor(0.3226, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)

---
## Optimizing Model

讓我們使用 `Pytorch` 官方的教學來快速優化一個模型，並且認識優化過程中的幾個重要元素。

In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# 下載訓練資料
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 下載驗證資料
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# 讀取資料，直接產生對應格式的批次資料
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 以下開始定義模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

定義超參數:

In [49]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss function 是用來衡量模型的好壞，根據任務類型的不同，我們會選擇不同的衡量方式

In [50]:
loss_fn = nn.CrossEntropyLoss()

優化器會根據 `loss` 來修正模型，根據任務類型的不同，我們會選擇不同的優化器及超參數:

In [51]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

定義訓練流程: 

In [52]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    # 重點1. 透過 data loader 取出資料
    for batch, (X, y) in enumerate(dataloader):
        # 重點2. 透過Forward Propagation 取得模型的輸出值
        pred = model(X)
        # 重點3. 計算模型表現
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

開始訓練!

In [53]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299525  [    0/60000]
loss: 2.294510  [ 6400/60000]
loss: 2.285777  [12800/60000]
loss: 2.281312  [19200/60000]
loss: 2.268651  [25600/60000]
loss: 2.271407  [32000/60000]
loss: 2.258977  [38400/60000]
loss: 2.255983  [44800/60000]
loss: 2.250064  [51200/60000]
loss: 2.227763  [57600/60000]
Epoch 2
-------------------------------
loss: 2.230824  [    0/60000]
loss: 2.215846  [ 6400/60000]
loss: 2.188242  [12800/60000]
loss: 2.191680  [19200/60000]
loss: 2.162056  [25600/60000]
loss: 2.187027  [32000/60000]
loss: 2.156347  [38400/60000]
loss: 2.151922  [44800/60000]
loss: 2.155007  [51200/60000]
loss: 2.107512  [57600/60000]
Epoch 3
-------------------------------
loss: 2.127699  [    0/60000]
loss: 2.091128  [ 6400/60000]
loss: 2.029878  [12800/60000]
loss: 2.039113  [19200/60000]
loss: 1.988983  [25600/60000]
loss: 2.048799  [32000/60000]
loss: 1.985110  [38400/60000]
loss: 1.986332  [44800/60000]
loss: 2.007752  [51200/60000]
loss: 1.92

---
## 休息時間

你知道嗎? `Pytorch`分門別類地將各種模組都整理好了!

舉例來說 : 

+ `torch.nn.xxx`: 各種不同的Layer
+ `torch.optim.xxx`: 各種不同的優化器

仿照上方的模型架構，試著抽換其他的優化器，看看訓練過程會有什麼不同吧?